# ATENÇÃO
Caro corretor, eu me chamo Ester Quintino e estou realizando o meu projeto com o Rodrigo Gatti,sob autorização do  professor Fábio Miranda.No início, eu fazia dupla com o Pedro Santos. Entretanto, não deu para continuar, e eu passei a fazer dupla com o Rodrigo . Inicialmente, nosso (meu e do Rodrigo) era "trident", mas devido a erros q encontramos ao decorrer do desenvolvimento do trabalho, tivemos que mudar o produto e mudamos para "Doritos".

# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [51]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [52]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [53]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @QuintinoEster
#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [54]:
#Produto escolhido:
produto = 'Doritos'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [55]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [56]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [57]:
%matplotlib inline
from mpmath import mpf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('ggplot')
import mpmath

#dados_Treinamento = pd.read_excel('trident.xlsx', "Treinamento", sep=',')
dados_Teste = pd.read_excel('Doritos.xlsx', "Teste", sep=',')


In [58]:
#pegando os dados do treinamento
dados_Treinamento = pd.read_excel('Doritos.xlsx', sheetname="Treinamento")
dados_Treinamento = dados_Treinamento.loc[:,["Treinamento", "Classificacao"]]

Abaixo tem-se a probabilidade da aba Treinamento:

In [59]:
valuecounts_Treinamento = pd.DataFrame(dados_Treinamento.Classificacao.value_counts(True)) #probabilidade total irrelevante e neutro (relevante)
probabilidade_total_irrelevante = valuecounts_Treinamento.Classificacao[0] #probabilidade total irrelevante

probabilidade_total_neutra = valuecounts_Treinamento.Classificacao[1] #probabilidade total neutro (relevante)
quantidade_de_neutros = len(dados_Treinamento[dados_Treinamento.Classificacao=="r"]) #Quantidade de neutros

quantidade_de_irrelevantes = len(dados_Treinamento[dados_Treinamento.Classificacao=="i"]) #Quantidade de irrelevantes
valuecounts_Treinamento

,Classificacao
i,0.576667
r,0.423333


Percebe-se que a probabilidade da classificação "i" é maior, na aba Treinamento. 

Código que gera um "tweet" único:

In [60]:

#juntando tudo em um tweet gigante
soma_Treinamento = np.sum(dados_Treinamento.Treinamento + " ")
soma_Treinamentoirrelevante = np.sum(dados_Treinamento[dados_Treinamento.Classificacao == "i"].Treinamento + " ")
soma_Treinamentoneutro = np.sum(dados_Treinamento[dados_Treinamento.Classificacao == "r"].Treinamento + " ")

Código que elimina alguns caracteres:

In [61]:
#Tirar todos os caracteres desnecessários
import string

excluir= set(string.punctuation)
def excluirpontos(soma):
    soma = soma.split()
    lista_palavrastweets = []
    for r in soma:
        for i in r:
            if i in excluir:
                r=r.replace(i, '')
        lista_palavrastweets.append(r)
    return lista_palavrastweets

treinamentototal = excluirpontos(soma_Treinamento)
treinamentoir = excluirpontos(soma_Treinamentoirrelevante)
treinamentone = excluirpontos(soma_Treinamentoneutro)

Transformação do "Tweet gigante(único)" em uma Serie :

In [62]:
#Tweet gigante virando uma Series (tabela)
treinamentototal= pd.Series(treinamentototal)
valuecounts_treinamentototal= treinamentototal.value_counts()

treinamentoir= pd.Series(treinamentoir)
valuecounts_treinamentoir= treinamentoir.value_counts()

treinamentone= pd.Series(treinamentone)
valuecounts_treinamentone= treinamentone.value_counts()

Calculando a Probabilidade das palavras Irrelevantes e das palavras Neutras(Relevantes):

In [63]:
prob_palavras_ir= []
prob_palavra= (valuecounts_treinamentoir + 1)/(len(valuecounts_treinamentototal) + len(valuecounts_treinamentoir))
prob_palavras_ir.append(prob_palavra)
print(prob_palavras_ir)

[doritos           0.049636
de                0.022313
eu                0.017760
o                 0.017760
que               0.015938
e                 0.015938
a                 0.014572
um                0.013206
do                0.010018
rt                0.009107
é                 0.008197
minha             0.007286
não               0.006831
no                0.006831
na                0.006375
pq                0.006375
as                0.005920
pra               0.005920
me                0.005920
meu               0.005464
dededoritos       0.005464
mas               0.005464
vou               0.005009
q                 0.005009
doritosmx         0.005009
vai               0.005009
em                0.004554
se                0.004554
uma               0.004554
vc                0.004098
                    ...   
lingua            0.000911
biscoito          0.000911
deus              0.000911
incógnita         0.000911
tengo             0.000911
limpar            0.000911


In [64]:
prob_palavras_ne= []
prob_palavra2= (valuecounts_treinamentone + 1)/(len(valuecounts_treinamentototal) + len(valuecounts_treinamentone))
prob_palavras_ne.append(prob_palavra2)
print(prob_palavras_ne)

[doritos        0.070407
de             0.035754
um             0.024752
e              0.020902
pra            0.020902
comer          0.017602
eu             0.015952
com            0.014851
que            0.011551
o              0.011001
rt             0.009901
é              0.009351
vontade        0.009351
queria         0.008801
meu            0.008801
ele            0.008251
no             0.007701
a              0.007151
uma            0.006051
vou            0.006051
minha          0.006051
coca           0.006051
do             0.006051
comprar        0.006051
agora          0.005501
mim            0.005501
               0.005501
da             0.004950
comendo        0.004950
me             0.004950
                 ...   
chocolate      0.001100
nunca          0.001100
frango         0.001100
bag            0.001100
perdi          0.001100
luto           0.001100
😍😍😍😍😍          0.001100
fumei          0.001100
caralho        0.001100
parar          0.001100
tanta          

Usando dados da aba Teste:

In [65]:
#pegando os dados do teste
dados_Teste = pd.read_excel('Doritos.xlsx', sheetname="Teste")
dados_Teste = dados_Teste.loc[:,["Teste", "Classificacao"]]

Código que elimina alguns caracteres:

In [77]:
lista_teste = dados_Teste.Teste
#tirando a pontuacao
import string
excluir= set(string.punctuation)

def excluirpontos(soma):
    lista_tweets_em_palavras=[]
    for s in range(len(soma)):
        frase = soma[s].split()
        lista_palavras=[]
        
        for r in frase:
            for l in r:
                if l in excluir:
                    r=r.replace(l,'')
            
            lista_palavras.append(r)
        lista_tweets_em_palavras.append(lista_palavras)
        
    return lista_tweets_em_palavras

lista_pal_tweets= excluirpontos(lista_teste)
#lista_pal_tweets

CÁLCULO DA PROBABILIDADE DO TWEET:

In [78]:
#calcula probabilidade da frase - neutro ou relevante
def prob(lista_tweets_inteiros,probabilidade_da_palavra,prob,tweets):
    lista_probabilidades = []
    for frase in lista_tweets_inteiros:
        probabilidade_frase = mpmath.mpf(1.000)
        for p in frase:
            if p in probabilidade_da_palavra:
                probabilidade_frase = probabilidade_frase * probabilidade_da_palavra[p] * prob
            else:
                probabilidade_frase = mpmath.mpf(1)/(tweets+len(treinamentototal.value_counts())) * probabilidade_frase * prob
        lista_probabilidades.append(probabilidade_frase)
    return lista_probabilidades

#cria lista sem pontuação
lista_tweets_em_palavras = excluirpontos(lista_teste)
#Cálculo da probabilidade do tweet ser neutro
teste_probabilidade_total_irrelevante  = prob(lista_tweets_em_palavras,prob_palavra,probabilidade_total_irrelevante ,quantidade_de_neutros)
#Cálculo da probabilidade do tweet ser relevante
teste_probabilidade_total_neutra = prob(lista_tweets_em_palavras,prob_palavra2,probabilidade_total_neutra,quantidade_de_irrelevantes)


#Compara as listas de probabilidade e devolve neutro ou relevante
def Comparação(teste_probabilidade_total_irrelevante,teste_probabilidade_total_neutra):
    resposta = []
    for i in range(len(teste_probabilidade_total_irrelevante)):
        if teste_probabilidade_total_irrelevante[i] > teste_probabilidade_total_neutra[i]:
            resposta.append('i')
        elif teste_probabilidade_total_irrelevante[i] < teste_probabilidade_total_neutra[i]:
            resposta.append('r')
    return resposta


#lista de respostas do aplicativo
resposta_teste = Comparação(teste_probabilidade_total_irrelevante,teste_probabilidade_total_neutra)

len(resposta_teste)


200

In [79]:
data = {'twitters': lista_tweets_em_palavras, 'Classificacao': dados_Teste.Classificacao, 'respostas': resposta_teste}


In [80]:
tabela_tweets_classificados = pd.DataFrame(data, columns = ['twitters','Classificacao','respostas'])
#tabelaT

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [81]:
lista_verificacao = []
for i in range(len(resposta_teste)):
    if dados_Teste.Classificacao[i] == resposta_teste[i]:
        if resposta_teste[i] == 'i':
            lista_verificacao.append('negativo verdadeiro')
        elif resposta_teste[i] == 'r':
            lista_verificacao.append('positivo verdadeiro')
            
    elif dados_Teste.Classificacao[i] != resposta_teste[i]:
        if resposta_teste[i] == 'i':
            lista_verificacao.append('negativo falso')
        elif resposta_teste[i] == 'r':
            lista_verificacao.append('positivo falso')


In [82]:
verificacaoS = pd.Series(lista_verificacao)
porcentagem = verificacaoS.value_counts(True)
porcentagem


negativo verdadeiro    0.490
negativo falso         0.290
positivo verdadeiro    0.205
positivo falso         0.015
dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


   Os resultados do teste com a planilha dos Doritos, mostram que a probabilidade de um twitter ser 
irrelevante é muito grande, o que influencia nos resultados do classificador. A porcentagem
de positivos falsos é 0.015 por esse motivo. Esse resultado tão baixo tem relação com sarcasmos e duplos negativos, que
não são corretamente interpretados pelo sistema,pelo fato de a formação da frase não ser considerada na conta.

   É possível expandir o projeto e pensar em reclamações mais específicas, como por exemplo, o mau cheiro do produto. Assim, é
possível organizar melhor a base de dados e obter resultados mais precisos.